In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("ps11.ipynb")

# Problem Set 11
## Logistic regression, automatic differentiation, and neural networks

In this problem set you will study binary classification using logistic regression, and implement neural networks

In [2]:
import sklearn
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.linear_model
import numpy as np
import torch 
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import matplotlib.pyplot as plt
rng_seed = 507
torch.manual_seed(rng_seed)

In [3]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
n, p = X.shape
n, p

(1797, 64)

## Question 1: Binary classification

In this exercise you will use `sklearn` to build a binary classifier.

Logistic regression assumes the probability model 

$$\mathbb{P}(y_i=1\mid \mathbf{x}_i) = \sigma(\mathbf{x}_i^T\boldsymbol{\beta}),$$ 

where $\mathbf{x}_i$ are rows of the data matrix $\mathbf{X}\in\mathbb{R}^{n\times p}$, $\mathbf{y}\in\{0,1\}^n$ is a vector of binary responses, and

$$\sigma(z)=\frac{1}{1+\exp(-z)}$$ 

is a *sigmoid* function which maps  real numbers into the interval $[0,1]$. 

In simple terms, a linear regression formula can be converted into a logistic regression by applying the sigmoid function. Using scikit-learn module, it gets very simple to apply these algorithms and that is what you will explore in this exercise.

The classifier will take as input a $28\times 28$ grayscale MNIST image, and return `1` if the image represents the number 5, and `0` otherwise.

*Note*: various algorithms implemented in `sklearn` are randomized. To utilize the same randomness as we did when generating the solutions (and hence, to ensure that your output passes the test cases), use `random_state=1` wherever necessary when calling `sklearn` methods.

**1(a)** (1 pt) Using the `mnist` data loaded above, create a standardized version of `X` where each column has zero mean and variance one. (Hint: use the `sklearn.preprocessing` module.)

In [4]:
stdscal = sklearn.preprocessing.StandardScaler()
Xs = stdscal.fit_transform(X)

In [5]:
grader.check("1a")

/opt/conda/lib/python3.11/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


1a results: All test cases passed!

**1(b)** (1 pt) Using the `mnist` data loaded above, create a vector `y5` which equals `1` if the the corresponding MINST image equals is of the number 5, and `0` otherwise.

In [6]:
y5 = (y==5).astype(np.int64)

In [7]:
grader.check("1b")

1b results: All test cases passed!

**1(c)**(1pt) Using `sklearn.model_selection.train_test_split`, divide the data into 70% training data and 30% test data. To ensure that your output matches our tests, pass the option `random_state=1` into the method.

In [8]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
                                    Xs, y5, test_size=0.3, random_state=1)

In [9]:
len(y5==1)

1797

In [10]:
grader.check("1c")

1c results: All test cases passed!

**1(d)**(2pt) Use `sklearn.linear_model.LogisticRegression` to train a binary classifier on the *training data only*. 

In [11]:
clf = sklearn.linear_model.LogisticRegression().fit(X_train, y_train)

In [12]:
grader.check("1d")

/opt/conda/lib/python3.11/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


1d results: All test cases passed!

<!-- BEGIN QUESTION -->

**1(e)**(1pt) How accurate is your trained classifier on `X_train`/`y_train`?  Use confusion matrix. 

Refer: 
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
* https://en.wikipedia.org/wiki/Confusion_matrix

Assign values to variables called TP, FP, FN, TN

In [13]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
TN, FP, FN, TP = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()

In [14]:
grader.check("1e")

/opt/conda/lib/python3.11/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


1e results: All test cases passed!

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**1(f)**(2pt) The regularization parameter can be varied by setting `LogisticRegression(C=C)` , where `C` is the value of the regularization penalty. What happens to the test error that you computed in the previous step as you vary `C`? Can you find a setting of `C` that results in lower test error than the default value `C=1`?

In this model, `C` is the inverse of regularization strength.   

Smaller `C` (stronger regularization) helps prevent overfitting by penalizing large coefficients. However, too much regularization may lead to underfitting, where the model is too simple and doesn't capture the underlying patterns in the data. So test error might increase compared to the default `C=1` due to underfitting.      

Larger `C` (weaker regularization) allows the model to fit the training data more closely. However, it may lead to overfitting, where the model captures noise in the training data that doesn't generalize well to new, unseen data. So test error might increase due to overfitting.        

Finding the optimal value of `C` involves tuning it through techniques like cross-validation. We can try different values of `C` and evaluate the model's performance using a validation set or through cross-validation. Typically, we believe the value of C that results in the lowest error on the validation set will lead to a low test error.

In practice, it's not guaranteed that there exists a setting of `C` that will always result in a lower test error than the default value of `C=1`. But utilizing the abovementioned method, it's very likely for us to find out such `C`. 

<!-- END QUESTION -->

## Question 2


**2(a)**(2pt) In this problem you will understand how to calculate derivatives using pytorch. 
**Note:** If you do not use pytorch tensors to solve these problems, hidden tests will fail.

Create a function called get_grad that receives a floating point value for 'x' and returns the gradient of the function at the given input value of 'x':

$$y = x^2 + 3x + 5$$



In [15]:
def get_grad(x):
    x = torch.tensor(x, dtype=torch.float, requires_grad=True)
    y = x**2 + 3*x + 5
    y.backward()
    return(x.grad)

In [16]:
grader.check("2a")

2a results: All test cases passed!

**2(b)**(2pt) 
We now extend the previous concept and get partial derivatives when you have two variables applied to the below function
$$𝑓(𝑢,𝑣)=𝑣𝑢+𝑢^3$$

Write a function 'get_grads' that takes in two floating point numbers corresponding to the two variables; u and v, and returns the gradients as a tuple with respect to the function.

In [17]:
def get_grads(u, v):
    u = torch.tensor(u, dtype=torch.float, requires_grad=True)
    v = torch.tensor(v, dtype=torch.float, requires_grad=True)
    f = u*v + u**3
    f.backward()
    return (u.grad, v.grad)

In [18]:
grader.check("2b")

2b results: All test cases passed!

**2(c)**(2pt) Extend the linear regression model from scratch shown in class by adding the bias 'b'
The linear function would be

$$y = w * x + b$$


In [19]:
X = torch.arange(-3, 3, 0.1).view(-1, 1)
f = 1 * X - 1
Y = f + 0.1 * torch.randn(X.size())
w = torch.tensor(-10.0, requires_grad = True)
b = torch.tensor(10.0, requires_grad = True)
lr = 0.1
loss_list = []

In [20]:
def criterion(yhat, y):
    return torch.mean((yhat - y) ** 2)

In [21]:
def forward(x):
    return w * x + b

In [22]:
def train_model(epochs, X, Y, lr):
    global w, b
    optimizer = torch.optim.SGD([w,b], lr=lr)
    for epoch in range (epochs):
        Yhat = forward(X)
        
        # calculate the loss per iteration
        loss = criterion(Yhat, Y)

        # store the loss at every iteration
        loss_list.append(loss.item())
        
        # backward pass: compute gradient 
        loss.backward()
        
        optimizer.step()
        #with torch.no_grad():
        #    w -= lr*w.grad
        #    b -= lr*b.grad
        w.grad.zero_()
        b.grad.zero_()

In [23]:
grader.check("2c")

2c results: All test cases passed!

**2(d)**(2pt)
In this problem, you will learn to use the deep learning framework PyTorch
We'll be using the Fashion MNIST dataset, which consists of 28x28 images that could be 10 different articles of clothing.


In [24]:
from torchvision import datasets as visiondata
training_data = visiondata.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

Run this cell to view a random sample from the training dataset.

In [25]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

<Figure size 800x800 with 0 Axes>

Here are some helper functions used for this assignment.

In [26]:
def train_loop(model, transform_fn, loss_fn, optimizer, dataloader, num_epochs):
    tbar = tqdm(range(num_epochs))
    for _ in tbar:
        loss_total = 0.
        for i, (x, y) in enumerate(dataloader):
            x = transform_fn(x)
            pred = model(x)
            loss = loss_fn(pred, y.squeeze(-1))
            ## Parameter updates
            model.zero_grad()
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
        tbar.set_description(f"Train loss: {loss_total/len(dataloader)}")
        
    return loss_total/len(dataloader)

In [27]:
def calculate_test_accuracy(model, transform_fn, test_dataloader):
    y_true = []
    y_pred = []
    tf = nn.Flatten()
    for (xi, yi) in test_dataloader:
        xi = transform_fn(xi)
        pred = model(xi)
        yi_pred = pred.argmax(-1)
        y_true.append(yi)
        y_pred.append(yi_pred)
    y_true = torch.cat(y_true, dim = 0)
    y_pred = torch.cat(y_pred, dim = 0)

    accuracy = (y_true == y_pred).float().mean()
    return accuracy

NN consists of an input layer, an activation function, and another output layer. Write a class called MultiClassNN that subclasses nn.Module. This module contains one attribute, net, which is an nn.Sequential object that is called on the .forward(x) method. Your task is to write the __init__() method to correctly construct net.

For example, if num_features=784, num_hidden=256, num_classes=10:

>>> mlp = MultiClassNN(28**2, 256, 10)
>>> mlp.net

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=256, out_features=10, bias=True)
  (3): LogSoftmax(dim=-1)
)

In [28]:
class MultiClassNN(nn.Module):
    def __init__(self, num_features, num_hidden, num_classes):
        """
        Arguments:
            num_features: The number of features in the input.
            num_hidden: Number of hidden features in the hidden layer:
            num_classes: Number of possible classes in the output
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=num_features, out_features= num_hidden, bias=True),
            nn.Sigmoid(),
            nn.Linear(in_features=num_hidden, out_features=num_classes, bias=True),
            nn.LogSoftmax(dim=-1)
        )
        
    def forward(self, x):
        return self.net(x)

In [29]:
grader.check("2d")

2d results: All test cases passed!

**2(e)**(1pt) 
Construct a `DataLoader` object of the Fashion MNIST training dataset.

In [30]:
batch_size = 64 
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

<!-- BEGIN QUESTION -->

**2(f)** 
(3pt) Initialize a `MultiClassNN` object called `mlp` and train it using the `train_loop()` function given at the beginning of the assignment (do not modify the `train_loop()` function). We will test your  `mlp` object on unseen test data.

Hints:
-  You need to initialize a `torch.optim.Optimizer` object for gradient descent. The standard choice is `torch.optim.Adam` with a learning rate `1e-3`.
-  You need to flatten the Fashion MNIST dataset to use within the `MultiClassNN`. This should be done with the `transform_fn` argument to `train_loop`. Try `nn.Flatten()`.
-  The output of `MultiClassNN` are the log probabilities of each class. To test the accuracy of your model, you should use the negative log-likelihood loss, `nn.NLLLoss()`, as loss function.

In [35]:
mlp = MultiClassNN(28**2, 256, 10)
mlp_optimizer = torch.optim.Adam(params=mlp.parameters(), lr=1e-3)
transform_fn = nn.Flatten()
loss_fn = nn.NLLLoss()
num_epochs = 20
loss_total = train_loop(mlp, transform_fn, loss_fn, mlp_optimizer,
                        train_dataloader, num_epochs)
print(loss_total)

  0%|          | 0/20 [00:00<?, ?it/s]

Train loss: 0.18537360186706472: 100%|██████████| 20/20 [02:24<00:00,  7.22s/it]

0.18537360186706472


<!-- END QUESTION -->



In [36]:
#test
test_dataloader = DataLoader(dataset=visiondata.FashionMNIST(
                                            root="data",
                                            train=False,
                                            download=True,
                                            transform=ToTensor()
                                        ),
                             batch_size=batch_size, shuffle=False)

mlp.train(False)
transform_fn.train(False)
loss_fn.train(False)

print(calculate_test_accuracy(mlp, transform_fn, test_dataloader))

tensor(0.8879)


## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [37]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)